<a href="https://colab.research.google.com/github/EhsanHonarvar/Practices/blob/main/4_ACT_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ACT New**

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re

In [3]:
!pip install opendatasets
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import opendatasets as od
import pandas
  
od.download(
    "https://www.kaggle.com/datasets/gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged")



# # بارگیری دیتاست
# data = pd.read_csv('Bitcoin_tweets_sentiment.csv', encoding='latin1')



Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: ehsanhonarvar
Your Kaggle Key: ··········


100%|██████████| 1.10G/1.10G [00:55<00:00, 21.4MB/s]


In [5]:
import pandas as pd

# reading the XLSX file
file =('bitcoin-tweets-16m-tweets-with-sentiment-tagged/mbsa.csv')
full_data = pd.read_csv(file)
  
# displaying the contents of the XLSX file
full_data.head()

,Date,text,Sentiment
0,2019-05-27,È appena uscito un nuovo video! LES CRYPTOMONN...,Positive
1,2019-05-27,Cardano: Digitize Currencies; EOS https://t.co...,Positive
2,2019-05-27,Another Test tweet that wasn't caught in the s...,Positive
3,2019-05-27,Current Crypto Prices! \n\nBTC: $8721.99 USD\n...,Positive
4,2019-05-27,Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...,Positive


In [6]:
data = full_data.sample(1000000)

data.head()
print('Total Sample: ',len(data))
print(data.groupby(data.Sentiment).count())

Total Sample:  1000000
             Date    text
Sentiment                
Negative   503153  503153
Neutral        24      24
Positive   450632  450632


In [7]:
# تعریف تابع پیش‌پردازش متن
def preprocess_text(text):
    text = re.sub(r'http\S+', '', str(text)) # حذف لینک‌ها
    text = re.sub(r'[^\w\s]','',text) # حذف نقطه، ویرگول و سایر نویسه‌های غیر حروف و اعداد
    text = re.sub(r'\d+', '', text) # حذف اعداد
    text = text.lower() # تبدیل به حروف کوچک
    return text

In [8]:
# پیش‌پردازش داده‌ها
data['text'] = data['text'].apply(preprocess_text)

In [9]:
# تبدیل برچسب‌ها به باینری (مثبت یا منفی) از نوع int
data['Sentiment'] = data['Sentiment'].apply(lambda x: 1 if (x =='positive' or x =='Positive') else 0)

# تقسیم داده‌ها به دو مجموعه آموزش و آزمون
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

In [10]:
# تبدیل متن به توکن‌ها
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
train_sequences = tokenizer.texts_to_sequences(train_data['text'])
test_sequences = tokenizer.texts_to_sequences(test_data['text'])

# پر کردن صفر در صورت نیاز
train_sequences = pad_sequences(train_sequences, padding='post')
test_sequences = pad_sequences(test_sequences, padding='post')

In [11]:
# تعریف پارامترهای شبکه
# MAX_SEQUENCE_LENGTH = len(max(train_sequences, key=len))
MAX_SEQUENCE_LENGTH = 20
VOCAB_SIZE = len(tokenizer.word_index) + 1
FILTERS = 64
KERNEL_SIZE = 3
DROPOUT_RATE = 0.5
NUM_HEADS = 8
D_MODEL = 64
NUM_LAYERS = 4
FFN_UNITS = 256
EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [12]:
# تعریف مدل
input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))
embedding_layer = tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=D_MODEL, input_length=MAX_SEQUENCE_LENGTH)(input_layer)
conv_layer = Conv1D(filters=FILTERS, kernel_size=KERNEL_SIZE, activation='gelu')(embedding_layer)
max_pooling_layer = MaxPooling1D(pool_size=2)(conv_layer)
flatten_layer = Flatten()(max_pooling_layer)
dropout_layer = Dropout(rate=DROPOUT_RATE)(flatten_layer)

In [13]:
######## اضافه شده 3


# تعریف لایه‌های ترانسفورمر
encoder_layers = []
for i in range(NUM_LAYERS):
  multi_head_attention_layer = MultiHeadAttention(num_heads=NUM_HEADS, key_dim=D_MODEL)(dropout_layer, dropout_layer, dropout_layer)
  add_attention_layer = Add()([multi_head_attention_layer, dropout_layer])
  layer_norm1 = LayerNormalization()(add_attention_layer)
  feed_forward_layer = Dense(units=FFN_UNITS, activation='gelu')(layer_norm1)
  add_feed_forward_layer = Add()([feed_forward_layer, layer_norm1])
  layer_norm2 = LayerNormalization()(add_feed_forward_layer)
  encoder_layers.append(layer_norm2)
  dropout_layer = Dropout(rate=DROPOUT_RATE)(layer_norm2)

# ترکیب لایه کانولوشنال با لایه‌های ترانسفورمر
concat_layer = tf.keras.layers.concatenate([flatten_layer]+encoder_layers)
dense_layer = Dense(units=64, activation='gelu', kernel_regularizer=l2(0.01))(concat_layer)
batch_norm_layer = BatchNormalization()(dense_layer)
output_layer = Dense(units=1, activation='linear')(batch_norm_layer)
output_layer = tf.keras.layers.Activation('softmax')(output_layer)

model = Model(inputs=input_layer, outputs=output_layer)


IndexError: ignored

In [ ]:
###### اضافه شده 2


# تعریف لایه‌های ترانسفورمر
encoder_layers = []
for i in range(NUM_LAYERS):
  multi_head_attention_layer = MultiHeadAttention(num_heads=NUM_HEADS, key_dim=D_MODEL)(dropout_layer, dropout_layer, dropout_layer)
  add_attention_layer = Add()([multi_head_attention_layer, dropout_layer])
  layer_norm1 = LayerNormalization()(add_attention_layer)
  feed_forward_layer = Dense(units=FFN_UNITS, activation='gelu')(layer_norm1)
  add_feed_forward_layer = Add()([feed_forward_layer, layer_norm1])
  layer_norm2 = LayerNormalization()(add_feed_forward_layer)
  encoder_layers.append(layer_norm2)
  dropout_layer = Dropout(rate=DROPOUT_RATE)(layer_norm2)



# # تعریف لایه‌های ترانسفورمر
# encoder_layers = []
# for i in range(NUM_LAYERS):
#   multi_head_attention_layer = MultiHeadAttention(num_heads=NUM_HEADS, key_dim=D_MODEL)(dropout_layer)
#   add_attention_layer = Add()([multi_head_attention_layer, dropout_layer])
#   layer_norm1 = LayerNormalization()(add_attention_layer)
#   feed_forward_layer = Dense(units=FFN_UNITS, activation='gelu')(layer_norm1)
#   add_feed_forward_layer = Add()([feed_forward_layer, layer_norm1])
#   layer_norm2 = LayerNormalization()(add_feed_forward_layer)
#   encoder_layers.append(layer_norm2)
#   dropout_layer = Dropout(rate=DROPOUT_RATE)(layer_norm2)

# ترکیب لایه کانولوشنال با لایه‌های ترانسفورمر
concat_layer = tf.keras.layers.concatenate([flatten_layer]+encoder_layers)
dense_layer = Dense(units=64, activation='gelu', kernel_regularizer=l2(0.01))(concat_layer)
batch_norm_layer = BatchNormalization()(dense_layer)
output_layer = Dense(units=1, activation='sigmoid')(batch_norm_layer)

model = Model(inputs=input_layer, outputs=output_layer)

IndexError: ignored

In [ ]:
# کامپایل مدل
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# آموزش مدل
history = model.fit(train_sequences, train_data['Sentiment'], epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(test_sequences, test_data['Sentiment']))

In [ ]:
# ارزیابی مدل
test_loss, test_acc = model.evaluate(test_sequences, test_data['Sentiment'])
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)